In [1]:
!pip install -U sagemaker


  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 36.5 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sagemaker.huggingface import HuggingFace
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = sagemaker.get_execution_role()

In [4]:
role

'arn:aws:iam::364819544619:role/SageMakerLLMRole'

In [5]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [6]:
# estimator = HuggingFace(
#     entry_point="train.py",
#     source_dir="./script",
#     role=role,
#     transformers_version="4.36",
#     pytorch_version="2.1",
#     py_version="py310",
#     instance_type="ml.g5.xlarge",
#     instance_count=1,
#     hyperparameters=hyperparameters
# )
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./script",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-ajithnarayanan/models/",
    hyperparameters=hyperparameters
)

In [7]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-ajithnarayanan/datasets/"
    # "train": "s3://llm-finetune-dataset-sunny/datasets/"
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-23-09-20-53-844


2025-11-23 09:20:54 Starting - Starting the training job...
2025-11-23 09:21:10 Pending - Training job waiting for capacity...
2025-11-23 09:21:48 Downloading - Downloading input data...
2025-11-23 09:22:13 Downloading - Downloading the training image........................
2025-11-23 09:26:10 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleD

In [8]:
estimator.model_data

's3://llm-model-artifacts-ajithnarayanan/models/huggingface-pytorch-training-2025-11-23-09-20-53-844/output/model.tar.gz'

In [9]:
# predictor = huggingface_estimator.deploy(
#     initial_instance_count=1,
#     instance_type="ml.g5.xlarge"
# )

# predictor.predict({"inputs": "Explain AWS S3"})


In [12]:
# from sagemaker import image_uris

# image_uris.retrieve(
#     framework="huggingface",
#     region="eu-north-1",   # change your region
#     version="4.37.0",
#     image_scope="inference"
# )